# Antenna reader example

In [ ]:
from radiocalibrationtoolkit import *
from healpy.newvisufunc import projview

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# create antenna instance
antenna_inst = AntennaPattern("./antenna_setup_files/SALLA_EW.xml")

In [ ]:
# XML values in dictionary
# antenna_inst.get_raw()

In [ ]:
# get antenna gain for unpolarized emission
df = antenna_inst.get(frequency=45, quantity="absolute")

# plot
fig = px.imshow(df.T.iloc[::-1, :], width=600, aspect="cube")
fig.update_layout(
    xaxis=dict(title="<b>azimuth</b>", tickprefix="<b>", ticksuffix="</b>", dtick=30),
    yaxis=dict(
        title="<b>zenith angle</b>",
        tickprefix="<b>",
        ticksuffix="</b>",
        range=(0, 90),
        tick0=0,
        dtick=10,
        autorange=False,
    ),
    coloraxis=dict(
        colorbar=dict(
            title=dict(
                text="<b>VEL</b>",
                side="right",
            ),
        tickprefix="<b>",
        ticksuffix="</b>",
        ),
    ),
    font=dict(
        size=15,
        color="black",
    ),
)

fig.show()

In [ ]:
# interpolate
df = antenna_inst.get(frequency=45, quantity='absolute', interp_phi=np.linspace(0, 360, 200), interp_theta=np.linspace(0,90, 100))

# plot
fig = px.imshow(df.T.iloc[::-1, :], width=600, aspect="cube")
fig.update_layout(
    xaxis=dict(title="<b>azimuth</b>", tickprefix="<b>", ticksuffix="</b>", dtick=30),
    yaxis=dict(
        title="<b>zenith angle</b>",
        tickprefix="<b>",
        ticksuffix="</b>",
        range=(0, 90),
        tick0=0,
        dtick=10,
        autorange=False,
    ),
    coloraxis=dict(
        colorbar=dict(
            title=dict(
                text="<b>VEL</b>",
                side="right",
            ),
        tickprefix="<b>",
        ticksuffix="</b>",
        ),
    ),
    font=dict(
        size=15,
        color="black",
    ),
)

In [ ]:
# to healpy
update_antenna_conventions={
        'shift_phi':-90, 
        'flip_theta':True, 
        'flip_phi':False,
        'in_degrees':True,
        'add_invisible_sky':True
    }

antenna_hpmap_inst = antenna_inst.convert2hp(frequency=45, **update_antenna_conventions)

In [ ]:
# hp maps are by default in galactic coordinates, so we need to specify the LST and latitude of the local observer
lst = 18
LATITUDE = -35.206667
rotation_parameters = create_rotation_parameters(lst, LATITUDE)
rotator = create_rotator(lst, LATITUDE, coord=["G", "C"])

In [ ]:
antenna_hpmap = antenna_hpmap_inst.get_map(rotator=rotator)

In [ ]:
# galaxy
projview(
    antenna_hpmap,
    cmap='jet',
    return_only_data=False,
    graticule=True,
    graticule_labels=True,
    title='Galactic coordinates',
    xtick_label_color='w',
    # projection_type='cart'
)

# from galaxy to local
projview(
    antenna_hpmap,
    cmap='jet',
    return_only_data=False,
    coord=['G','C'],
    rot=rotation_parameters,
    graticule=True,
    graticule_labels=True,
    title='Local coordinates',
    xtick_label_color='w',
    # projection_type='cart'
)